In [1]:
import pandas as pd
import numpy as np

In [5]:
!unzip train.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  test_labels.csv.zip
replace test_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
test_labels_df = pd.read_csv('test_labels.csv')
test_labels_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [3]:
#X_train = train_df['comment_text']
small_X_train = train_df['comment_text']
small_X_train = small_X_train.tolist()
small_X_train = small_X_train[:3000]

In [12]:
print(len(small_X_train))

3000


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
import torch.nn.functional as F

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("text-classification",model = model, tokenizer = tokenizer)
#results = classifier()

# tokens = tokenizer.tokenize()
# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# input_ids = tokenizer()

batch = tokenizer(small_X_train, padding=True,truncation=True,max_length=100, return_tensors="pt")

In [6]:
with torch.no_grad():
  outputs = model(**batch)
  print(outputs)
  predictions = F.softmax(outputs.logits, dim = 1)
  print(predictions)
  labels = torch.argmax(predictions, dim=1)
  print(labels)
  labels = [model.config.id2label[label_id] for label_id in labels.tolist()]

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.8418, -3.1863],
        [ 2.4611, -2.0358],
        [ 3.4885, -2.8373],
        ...,
        [ 2.5870, -2.0933],
        [ 2.1544, -1.7460],
        [ 2.1635, -1.6817]]), hidden_states=None, attentions=None)
tensor([[9.9911e-01, 8.8582e-04],
        [9.8898e-01, 1.1020e-02],
        [9.9821e-01, 1.7864e-03],
        ...,
        [9.9081e-01, 9.1914e-03],
        [9.8017e-01, 1.9832e-02],
        [9.7907e-01, 2.0935e-02]])
tensor([0, 0, 0,  ..., 0, 0, 0])


In [7]:
save_directory = "saved"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [8]:

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory)

In [21]:
list_test = test_df['comment_text'].tolist()[0:3]
results = classifier(list_test)
print(results)

[{'label': 'NEGATIVE', 'score': 0.5376824140548706}, {'label': 'POSITIVE', 'score': 0.9835041761398315}, {'label': 'NEGATIVE', 'score': 0.9680620431900024}]
